# Zonal Heatmaps

In this notebook, heatmaps are generated for each SBB zone to predict the sale of subscriptions. The features such as income, age etc. are converted into heatmaps and these images are used to predict the subscription sale. 

Importing some useful libs to be used in processing

In [1]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

In [2]:
from skimage.io import imread
from io import BytesIO
from tqdm.notebook import tqdm
import logging
import skimage
from collections import defaultdict
from scipy import stats
from scipy.linalg import LinAlgError
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from skimage import color
from PIL import Image

# Load Datasets

4 datasets are loaded:
- A dataset with workplace info per hectare cell
- A dataset for Zones and their geometry (Polygons/Multipoligons) 
- A dataset for zones along with number of employmenst and subscriptions
- An anonymized data set for persons for the demographinc info, along with their location and zone

### Hectar data

In [3]:
hectar_path = Path('Data/STATENT2017_N08_V190822.csv')
hectars = pd.read_csv(hectar_path, usecols=['B1708EMPT', 'E_KOORD', 'N_KOORD', 'X_KOORD', 'Y_KOORD'])
hectars.shape

(216100, 5)

Verify coordinates are hectare cells. A hectare cell is 100 * 100 block of all coordinates within it, like a grid on the map. 

In [4]:
hectars['E_KOORD'].mod(100).eq(0).all()

True

In [5]:
hectars = gpd.GeoDataFrame(hectars, geometry=gpd.points_from_xy(hectars['E_KOORD'], hectars['N_KOORD']))
type(hectars)

geopandas.geodataframe.GeoDataFrame

In [6]:
#hectars = hectars.set_crs(epsg=2056)

In [7]:
hectars.head(5)

,E_KOORD,N_KOORD,X_KOORD,Y_KOORD,B1708EMPT,geometry
0,2486200,1111300,486200,111300,3,POINT (2486200.000 1111300.000)
1,2486200,1111500,486200,111500,3,POINT (2486200.000 1111500.000)
2,2486300,1111700,486300,111700,3,POINT (2486300.000 1111700.000)
3,2486400,1111700,486400,111700,7,POINT (2486400.000 1111700.000)
4,2486500,1111600,486500,111600,3,POINT (2486500.000 1111600.000)


### Zone Data

In [8]:
geo_path = Path('data/Verkehrszonen_Schweiz_NPVM_2017.shp')
zones = gpd.read_file(geo_path)
zones.head(5)

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
0,101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2678311.335 1235001.742, 2678311.038..."
1,201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2674392.852 1239006.422, 2674436.739..."
2,201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2676361.095 1235716.687, 2676358.928..."
3,201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2675639.621 1236730.880, 2675624.262..."
4,201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2676518.641 1237003.354, 2676524.234..."


### Strukturdaten

In [9]:
zones_path = Path('data/1100_2017-Strukturdaten_Data_D_F_v1.1.0.xlsx')
structure_data = pd.read_excel(zones_path, skiprows=2, index_col=0, na_values=['X']).drop(['N_Gem', 'stg_type'], axis=1)
structure_data.head(5)

,munid,msrid,cantid,r,re,r_age,R_0017_CARNOTC,R_1824_CARNOTC,R_2544_CARNOTC,R_4564_CARNOTC,...,pupils_II,e_a,students,airport_passengers,e_e,e_m,e_cb,fte_e,fte_m,fte_cb
npvmid,,,,,,,,,,,,,,,,,,,,,
101001,1,4,1,1949.0,1173.0,43.0,0.0,78.0,308.0,558.0,...,0,7,0,0,211,254,0,126,184,0
201001,2,4,1,1238.0,746.0,40.0,0.0,59.0,246.0,296.0,...,0,11,0,0,160,141,0,106,110,0
201002,2,4,1,407.0,139.0,49.0,0.0,10.0,69.0,74.0,...,0,84,0,0,1688,822,10,1305,738,8
201003,2,4,1,1314.0,721.0,44.0,0.0,41.0,200.0,279.0,...,0,26,0,0,246,129,1,166,103,1
201004,2,4,1,1195.0,564.0,44.0,0.0,36.0,197.0,172.0,...,0,34,0,0,568,335,0,419,279,0


Lets start merging the 3 loaded datasets together.

fisrt we merge the Structure data (zone features) with zone geo data
From feature data, we only select following
- Number of employments
- Number of subscriptions, 3 different types

In [10]:
cols = ['sum_e',   # Number of employements
        'R_HT',    # Number of Halbtax
        'R_GATC',  # Number of GA
        'R_LTC']   # Number of Regional passes

In [11]:
zones_info = zones.set_index('ID').join(structure_data[cols])
zones_info[cols] = zones_info[cols].fillna(0)
zones_info.head(5)

,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry,sum_e,R_HT,R_GATC,R_LTC
ID,,,,,,,,,,,,,,,,,,
101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2678311.335 1235001.742, 2678311.038...",465,754.0,189.0,251.0
201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2674392.852 1239006.422, 2674436.739...",301,396.0,53.0,173.0
201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2676361.095 1235716.687, 2676358.928...",2520,157.0,54.0,63.0
201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2675639.621 1236730.880, 2675624.262...",376,520.0,87.0,188.0
201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,DietikonSchlieren,"POLYGON ((2676518.641 1237003.354, 2676524.234...",903,373.0,117.0,196.0


Not we add the hectare information plus the number of workplaces, the 1st dataset we loaded

In [12]:
hectar_zone = gpd.sjoin(hectars[['E_KOORD', 'N_KOORD', 'geometry', 'B1708EMPT']], zones_info[['N_Gem','N_AMR','N_Agglo','geometry']+cols].reset_index(), how="right").drop(['index_left'], axis=1).set_index('ID')
hectar_zone.head(1)

C:\Users\maria\anaconda3\lib\site-packages\geopandas\tools\sjoin.py:58: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:2056'})
  warn(


,E_KOORD,N_KOORD,B1708EMPT,N_Gem,N_AMR,N_Agglo,geometry,sum_e,R_HT,R_GATC,R_LTC
ID,,,,,,,,,,,
101001,2679600.0,1236000.0,33.0,Aeugst am Albis,DietikonSchlieren,Zürich,"POLYGON ((2678311.335 1235001.742, 2678311.038...",465,754.0,189.0,251.0


merge all subscriptions to one sum

In [13]:
cols = ['R_HT','R_GATC','R_LTC']
hectar_zone['sum_s'] = hectar_zone[cols].sum(axis=1).astype(int)
hectar_zone.drop(cols, axis=1, inplace=True)
hectar_zone.head(5)

,E_KOORD,N_KOORD,B1708EMPT,N_Gem,N_AMR,N_Agglo,geometry,sum_e,sum_s
ID,,,,,,,,,
101001,2679600.0,1236000.0,33.0,Aeugst am Albis,DietikonSchlieren,Zürich,"POLYGON ((2678311.335 1235001.742, 2678311.038...",465,1194
101001,2679700.0,1236400.0,3.0,Aeugst am Albis,DietikonSchlieren,Zürich,"POLYGON ((2678311.335 1235001.742, 2678311.038...",465,1194
101001,2680500.0,1236800.0,6.0,Aeugst am Albis,DietikonSchlieren,Zürich,"POLYGON ((2678311.335 1235001.742, 2678311.038...",465,1194
101001,2680500.0,1236900.0,3.0,Aeugst am Albis,DietikonSchlieren,Zürich,"POLYGON ((2678311.335 1235001.742, 2678311.038...",465,1194
101001,2680600.0,1236900.0,3.0,Aeugst am Albis,DietikonSchlieren,Zürich,"POLYGON ((2678311.335 1235001.742, 2678311.038...",465,1194


### SynPop

Finally we load the anonymed people data 

In [14]:
synpop_path = Path('data/synpop-anonymisiert.7z')

In [15]:
synpop = pd.read_csv('data/synpop-anonymisiert.csv', sep=';', index_col=0)
synpop.head()

C:\Users\maria\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\maria\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,person_id,sex,age,nation,child_in_household,household_size,language,level_of_employment,household_income,education,position_in_edu,position_in_bus,car_ownership,car_company,mobility,zone_id,xcoord,ycoord,ID_AggZone1500
0,1,F,25-44,non-swiss,True,3-4,french,80-100,145+,uni,NaN,employee,True,False,car,220601002,2578019.0,1181293.0,ID_alt_2206
1,946027,M,0-17,non-swiss,True,3-4,french,0,145+,no_edu,NaN,unemployed,False,False,NaN,220601002,2578069.0,1181345.0,ID_alt_2206
2,1569416,M,25-44,non-swiss,True,3-4,french,80-100,145+,higher_edu,NaN,ceo,True,False,car,220601003,2578051.0,1181435.0,ID_alt_2206
3,4351561,F,0-17,non-swiss,True,3-4,french,0,145+,no_edu,NaN,unemployed,False,False,NaN,220601002,2577996.0,1181237.0,ID_alt_2206
4,2,M,45-64,non-swiss,False,2,other,0,50-95k,no_edu,NaN,NaN,False,False,car & va,24301016,2672710.0,1251223.0,ID_alt_243


In [16]:
synpop = synpop.fillna({'position_in_edu': 'null',
                       'position_in_bus': 'null',
                       'mobility': 'null'})

Columns to take as features

Columns to take as Index

Columns to take as Target

In [17]:
cols = ['sex', 'age', 'nation', 'household_size', 'language', 'level_of_employment', 'household_income', 'education', 'position_in_edu', 'position_in_bus']#, 'mobility']
idx= ['zone_id','person_id','xcoord', 'ycoord']
tgt= ['sum_s']

In [18]:
synpop = synpop.astype({c: 'category' for c in cols})

#### Features

In [19]:
synpop = synpop[idx+cols]
synpop.head(2)

,zone_id,person_id,xcoord,ycoord,sex,age,nation,household_size,language,level_of_employment,household_income,education,position_in_edu,position_in_bus
0,220601002,1,2578019.0,1181293.0,F,25-44,non-swiss,3-4,french,80-100,145+,uni,null,employee
1,220601002,946027,2578069.0,1181345.0,M,0-17,non-swiss,3-4,french,0,145+,no_edu,null,unemployed


#### Target

In [20]:
zone_subs = hectar_zone.reset_index()[['ID']+tgt].drop_duplicates(subset=['ID']+tgt).reset_index().drop('index', axis=1)
zone_subs.head(2)

,ID,sum_s
0,101001,1194
1,201001,622


In [21]:
zone_subs.to_csv('target.csv',index=False)

### Generate images per zone

Tries to calculate a density heatmap based on individual X,Y coordinates, unless at a particular zone there are less than 5 persons for the same category. In case of exception, a grayed image is produced with no density.

In [22]:
def plot_contour(synpop, levels, ax=None, vmin=None, vmax=None, resolution=100):
    data = synpop[['xcoord', 'ycoord']]
    skip = False
    xx=np.zeros((2, 2)).astype(float)
    yy=np.zeros((2, 2)).astype(float)
    density = np.zeros((2, 2)).astype(float)
    # min 5 persons to generate heatmap
    if len(data) < 5:
        skip = True
    else:
        try:
            xx, yy = np.mgrid[data['xcoord'].min():data['xcoord'].max():resolution, data['ycoord'].min():data['ycoord'].max():resolution]
            kde = stats.gaussian_kde([data['xcoord'], data['ycoord']])
            density = kde(np.c_[xx.flat, yy.flat].T).reshape(xx.shape)
        except Exception as e:
            skip = True

    ## Sometime get array of dim (x,1), Skip those cases
    if density.shape[0] < 2 or density.shape[1] < 2:
        skip = True
        

    if ax is not None:
        if skip:
            cset = ax.contourf(np.zeros((2,2)), cmap="gray")
        else:
            cset = ax.contourf(xx, yy, density, cmap="gray", extend='both', levels=levels, vmin=vmin, vmax=vmax)
        ax.axis('off')
    return density

Util Function to get 128 x 128 heatmap using abive density function

In [23]:
#from PIL import Image
#not only saves but also returns image as an array
def get_image(df, filename, max_level):
    # plot
    wh = 128

    fig = plt.figure(frameon=False)
    fig.set_size_inches(wh, wh)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    levels = np.linspace(0, max_level, 50)
    _ = plot_contour(df, levels=levels, ax=ax, resolution=50)
    
    fig.savefig(filename, dpi=1)
    
    image = Image.open(f'{filename}.png').convert('L').resize((128,128), Image.ANTIALIAS)
    
    plt.close()
    return np.asarray(image)

Get max column values from a json

In [24]:
col_maxs = defaultdict(dict)
import json
with open('data/col_maxs.json') as f:
    col_maxs = json.load(f)
#col_maxs = { col: col_maxs[col] for col in cols }
col_maxs.values()

dict_values([{'F': 2.8274611241487096e-05, 'M': 8.782742643018475e-05}, {'0-17': 8.311106602748824e-06, '18-24': 1.2495678108475149e-06, '25-44': 8.9468481948856e-05, '45-64': 8.087862677469907e-05, '65-74': 2.084143057425632e-05, '75+': 2.189179976319869e-05}, {'non-swiss': 8.87129731100414e-05, 'swiss': 2.0794490656125187e-06}, {'1': 1.6161959131292892e-05, '2': 4.358957697653426e-06, '3-4': 5.409114907087332e-07, '5+': 1.0093211442898072e-05}, {'french': 2.9015641361967596e-06, 'german': 3.822987803256064e-06, 'italian': 1.3550174533042395e-06, 'other': 7.850848303513828e-07, 'romansh': 5.127696474901245e-06}, {'0': 8.644713510379517e-07, '1-39': 6.65051105939947e-06, '40-79': 6.14492367368526e-06, '80-100': 6.373470453885874e-06}, {'0-50k': 2.740849671608553e-07, '145+': 9.83070701671678e-06, '50-95k': 1.0169202360407544e-06, '95-145k': 8.52786037375578e-05}, {'higher_edu': 5.185263192680788e-06, 'no_edu': 1.8662386266820887e-06, 'secondary': 8.4803054671217e-06, 'uni': 6.697172178

Aggregations per zone

In [25]:
%%time
col_counts = synpop[['zone_id']+cols].set_index('zone_id').stack().groupby(level=[0,1]).value_counts().unstack(level=[1,2]).fillna(0)[cols]

Wall time: 3min 5s


In [26]:
col_counts

sex           age                                     nation  \
               F      M  45-64  25-44   0-17  65-74  18-24    75+   swiss   
zone_id                                                                     
101001     977.0  972.0  705.0  404.0  343.0  213.0  150.0  134.0  1694.0   
201001     636.0  630.0  329.0  359.0  245.0  114.0  102.0  117.0   919.0   
201002     213.0  208.0  108.0  123.0   76.0   41.0   46.0   27.0   301.0   
201003     629.0  646.0  356.0  351.0  230.0  125.0   99.0  114.0   926.0   
201004     600.0  645.0  338.0  336.0  247.0  129.0   96.0   99.0   884.0   
...          ...    ...    ...    ...    ...    ...    ...    ...     ...   
681001001  225.0  199.0  107.0   82.0   74.0   71.0   38.0   52.0   398.0   
681001002   78.0   74.0   48.0   34.0   33.0   16.0    8.0   13.0   136.0   
681001003   88.0   97.0   47.0   41.0   42.0   25.0   13.0   17.0   175.0   
681001004  166.0  146.0   96.0   63.0   59.0   38.0   20.0   36.0   287.0   
681001005   46.0   58.0   32.0   21.0   29.0    9.0    4.0    9.0    92.0   

                     ... position_in_edu                position_in_bus  \
          non-swiss  ...            null  pupil student        employee   
zone_id              ...                                                  
101001        255.0  ...          1587.0  291.0    71.0           570.0   
201001        347.0  ...          1062.0  175.0    29.0           443.0   
201002        120.0  ...           353.0   55.0    13.0           151.0   
201003        349.0  ...          1107.0  139.0    29.0           415.0   
201004        361.0  ...          1053.0  162.0    30.0           402.0   
...             ...  ...             ...    ...     ...             ...   
681001001      26.0  ...           343.0   69.0    12.0           103.0   
681001002      16.0  ...           131.0   20.0     1.0            30.0   
681001003      10.0  ...           146.0   34.0     5.0            34.0   
681001004      25.0  ...           262.0   42.0     8.0            65.0   
681001005      12.0  ...            85.0   19.0     0.0            28.0   

                                                                         
            null    ceo unemployed management apprentice bus_management  
zone_id                                                                  
101001     494.0  343.0      270.0      192.0       60.0           20.0  
201001     349.0  119.0      191.0      110.0       41.0           13.0  
201002     120.0   38.0       52.0       39.0       20.0            1.0  
201003     373.0  153.0      172.0      119.0       39.0            4.0  
201004     370.0  135.0      188.0      102.0       39.0            9.0  
...          ...    ...        ...        ...        ...            ...  
681001001  172.0   59.0       64.0       12.0       12.0            2.0  
681001002   55.0   27.0       26.0       10.0        3.0            1.0  
681001003   65.0   33.0       38.0       11.0        4.0            0.0  
681001004  115.0   51.0       49.0       21.0       10.0            1.0  
681001005   28.0   15.0       24.0        4.0        5.0            0.0  

[7914 rows x 43 columns]

In [27]:
col_cats = list(set([(str(x),str(y)) for x,y in list(col_counts.columns)])) #column categories
col_cats.sort()
len(col_cats)

41

In [ ]:
%%time
col_counts = synpop[idx+cols].set_index(idx).stack().groupby(level=[0,1,2,3,4]).value_counts().unstack(level=[4,5]).fillna(0)[cols].reset_index()
#.stack().groupby(level=[0,1]).value_counts().unstack(level=[1,2]).fillna(0)

In [ ]:
col_counts

In [ ]:
failed = []

cnt = 1
tot = len(list(zones.ID))
zpath = Path('zonal_heatmaps')
zpath.mkdir(exist_ok=True, parents=True)

path = Path('heatmap_images')
for ID in list(zones.ID):
    print(f"Zone Id {ID} ", end= " ")
    images = np.array([])
    try:
        # get data. zone_df is a new dataframe with matched ID
        #zone_df subset of col_count for one specific zone
        #person_df: col_count's each column and each category
        zone_df = col_counts[col_counts.zone_id==ID]
        for col,cat in col_cats:#in [('language','romansh')]:
        #for col,cat in [('language','romansh')]:    column and category
            
            person_df = zone_df[zone_df[col][cat]==1].set_index('person_id')[['xcoord','ycoord']].dropna().astype(int)
            path_cat = path.joinpath(str(col)).joinpath(str(cat))
            path_cat.mkdir(exist_ok=True, parents=True)
            img_path = path_cat.joinpath(str(ID))
            
            try:
                image = get_image(person_df, img_path, col_maxs[col][cat])
            except:
                image = np.zeros((128,128))
            
            #First image, array is zero
            if images.size == 0:
                images = np.copy(image)
            else:
                images = np.concatenate((images,image),axis=0)
            
            plt.close()
    except Exception as e:
        failed.append(ID)
        logging.error(e, exc_info=True)
        print(f'Error in Zone: {ID} {col} {cat} , Data Len:{len(person_df)}', end = "")
        
    Image.fromarray(images).convert("L").save(f'{zpath}/{ID}.png') #long images stored in zpath
    print(f"{cnt}/{tot} done")  
    cnt+=1

### Neural Network

In [ ]:
from tensorflow import keras 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from matplotlib.pyplot import imshow
from PIL import Image

import tensorflow as tf
import glob #lists all filenames for a directory

In [ ]:
batch_size = 16
train_data_dir= 'zonal_heatmaps'
train_targets = 'target.csv'

img_width=128
img_height=5504

In [ ]:


img = load_img('zonal_heatmaps/201001.png').convert('L')  # this is a PIL image (1, 5504, 128)
x = img_to_array(img)     # this is a Numpy array with shape (5504, 128, 1)
print(x.shape)            # (5504, 128, 1)
x = np.moveaxis(x, -1, 0) # (1, 5504, 128)
w = 128
ch = int(x.shape[1]/128)  # 43
x = x.reshape(ch, w, w)   # (43, 128, 128)
x = x.reshape((1,) + x.shape) # (1, 43, 128, 128)
x.shape

In [ ]:
plt.figure(figsize = (200,200))
imshow(img, cmap='gray')

In [ ]:
def changeShape(image):
    print(image.shape)
    #print(image.shape)
    image = image.reshape(ch, w, w)
    print(image.shape)
    
    return image

#data augementation
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15
                                   #,preprocessing_function=changeShape
                                  )

In [ ]:
train_generator = train_datagen.flow_from_directory('.', 
    classes=[train_data_dir],
    batch_size=batch_size,
    subset='training')

In [ ]:

validation_generator = train_datagen.flow_from_directory('.', 
    classes=[train_data_dir],
    #color_mode='grayscale',
    batch_size=batch_size,
    subset='validation') # set as validation data

In [ ]:
train_lst = []
train_files = []

## 'zonal_images\\101001.png'
for file in train_generator.filenames:
    train_lst.append(int(str(file).replace(train_data_dir+'\\','').replace('.png','')))
train_lst.sort() #zones list

for file in train_generator.filenames:
    train_files.append(str(file))
train_files.sort() #training file names

test_lst = []
test_files = []
for file in validation_generator.filenames:
    test_lst.append(int(str(file).replace(train_data_dir+'\\','').replace('.png','')))
test_lst.sort()

for file in validation_generator.filenames:
    test_files.append(str(file))
test_files.sort()

In [ ]:
labels = pd.read_csv(train_targets)

train_labels = list(labels[labels.ID.isin(train_lst)].sum_s)
test_labels = list(labels[labels.ID.isin(test_lst)].sum_s)

len(labels)

In [ ]:
train_images = []
test_images  = []

for filename in sorted(glob.glob(train_data_dir+"/*.png"), key=lambda item: int(item.replace(train_data_dir+"\\","").replace(".png",""))):
    if filename in train_files:
        im=Image.open(filename).convert('L')
        im = np.squeeze(img_to_array(im)).reshape(ch,w,w)
        train_images.append(im)
    if filename in test_files:
        im=Image.open(filename).convert('L')
        im = np.squeeze(img_to_array(im)).reshape(ch,w,w)
        test_images.append(im)

print(len(train_images))
print(len(test_images))
train_images[0].shape

In [ ]:
## convert to np.array
train_images = np.asarray(train_images)
train_labels = np.asarray(train_labels)
test_images = np.asarray(test_images)
test_labels = np.asarray(test_labels)

In [ ]:
from tensorflow import keras

def coeff_determination(y_true, y_pred):
    SS_res =  keras.backend.sum(keras.backend.square( y_true-y_pred ))
    SS_tot = keras.backend.sum(keras.backend.square( y_true - keras.backend.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + keras.backend.epsilon()) )

In [ ]:
model = Sequential()
model.add(Conv2D(32, (4, 4), input_shape=(ch, w, w))) #32 filters of 4x4
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('relu'))

model.summary()

In [ ]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=[tf.keras.metrics.RootMeanSquaredError(), coeff_determination])

In [ ]:
hist = model.fit(train_images, train_labels,
        epochs=100,
        batch_size=batch_size,
        validation_data=(test_images, test_labels))

model.save_weights('model.h5')

In [ ]:
# Train RMSE
hist.history['root_mean_squared_error'][-1]

In [ ]:
# Validation RMSE last value
hist.history['val_root_mean_squared_error'][-1]

In [ ]:
# Visualize history
# Plot history: Loss
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,10)
plt.plot(hist.history['root_mean_squared_error'])
plt.title('RMSE history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
# Visualize history
# Plot history: Loss
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,10)
plt.plot(hist.history['val_root_mean_squared_error'])
plt.title('VAL RMSE history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
### END